In [1]:
import warnings
warnings.filterwarnings('ignore')

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Lesson 4*
On Monday we talked about summarizing your documents using just token counts. Today, we're going to learn about a much more sophiscated approach - learning 'topics' from documents. Topics are a latent structure. They are not directly observable in the data, but we know they're there by reading them.

> **latent**: existing but not yet developed or manifest; hidden or concealed.

## Use Cases
Primary use case: what the hell are your documents about? Who might want to know that in industry - 
* Identifing common themes in customer reviews
* Discoverying the needle in a haystack 
* Monitoring communications (Email - State Department) 

## Learning Objectives
*At the end of the lesson you should be able to:*
* <a href="#p1">Part 1</a>: Describe how an LDA Model words
* <a href="#p2">Part 2</a>: Estimate a LDA Model with Gensim
* <a href="#p3">Part 3</a>: Interpret LDA results
* <a href="#p4">Part 4</a>: Select the appropriate number of topics

## Part 1: Latent Dirchilet Allocation (LDA) Models
<a id="#p1"></a>

LDA is a "generative probabilistic model". 

Let's play with a modoel available [here](https://lettier.com/projects/lda-topic-modeling/)

## Part 2: Estimating LDA Models with Gensim
<a id="#p1"></a>

### A Litterary Introduction: *Jane Austen V. Charlotte Bronte*
Despite being born nearly forty years apart, modern fans often pit Jane Austen & Charlotte Bronte against one another in a battle for litterary  supremacy. The battle centers around the topics of education for women, courting, and marriage. The authors' similiar backgrounds naturally draw comparisons, but the modern fascination is probably due to novelility of British women publishing novels during the early 19th century. 

Can we help close a litterary battle for supremacy and simply acknowledge that the authors addressed different topics and deserve to be acknowledged as excellent authors each in their own right?

We're going to apply Latent Dirichlet Allocation a machine learning alogrithm for topic modeling to each of the author's novels to compare the distribution of topics in their novels.

In [4]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

### Novel Data
I grabbed the novel data pre-split into a bunch of smaller chuncks

In [5]:
path = './data/austen-brontë-split'

### Text Preprocessing
**Challenge**: update the function `tokenize` with any technique you have learned so far this week. 

In [6]:
titles = [t[:-4] for t in os.listdir(path)]

In [9]:
titles[:5]

['Austen_Emma0026',
 'Austen_Emma0032',
 'CBronte_Villette0086',
 'CBronte_Jane0099',
 'CBronte_Villette0092']

In [10]:
STOPWORDS = set(STOPWORDS).union(set(['said', 'mr', 'mrs']))

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [11]:
tokenize("Hello World! This a test of the tokenization method")

['hello', 'world', 'test', 'tokenization', 'method']

### Author DataFrame


In [12]:
df = pd.DataFrame(index=titles)

In [13]:
df.head()

""
Austen_Emma0026
Austen_Emma0032
CBronte_Villette0086
CBronte_Jane0099
CBronte_Villette0092


In [14]:
df['author'] = df.reset_index()['index'].apply(lambda x: x.split('_')[0]).tolist()
df['book'] = df.reset_index()['index'].apply(lambda x: x.split('_')[1][:-4]).tolist()
df['section'] = df.reset_index()['index'].apply(lambda x: x[-4:]).tolist()
df['section'] = df['section'].astype('int')

In [15]:
df['author'] = df['author'].map({'Austen':1, 'CBronte':0})

In [16]:
df.author.value_counts()

0    441
1    372
Name: author, dtype: int64

### Streaming Documents
Here we use a new pythonic thingy: the `yield` statement in our fucntion. This allows us to iterate over a bunch of documents without actually reading them into memory. You can see how we use this fucntion later on. 

In [17]:
def doc_stream(path):
    for f in os.listdir(path):
        with open(os.path.join(path,f)) as t:
            text = t.read().strip('\n')
            tokens = tokenize(str(text))
            yield tokens

### Gensim LDA Topic Modeling

In [18]:
# A Dictionary Representation of all the words in our corpus
id2word = corpora.Dictionary(doc_stream(path))

In [19]:
# Let's remove extreme values from the dataset
id2word.filter_extremes(no_below=10, no_above=0.75)

In [20]:
# a bag of words(bow) representation of our corpus
# Note: we haven't actually read any text into memory here
corpus = [id2word.doc2bow(text) for text in doc_stream(path)]

In [21]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=4
                  )

In [22]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [23]:
topics = [' '.join(t[2:5]) for t in words]

## Part 3: Interpret LDA Results
<a id="#p3"></a>

### Topic Distance Visualization

In [24]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [25]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.083353 -0.002954       1        1  32.533401
4     -0.096960 -0.001643       2        1  18.099949
1     -0.142512  0.019571       3        1   9.962263
8     -0.124742  0.003269       4        1   8.912821
2      0.025857 -0.030816       5        1   6.869203
9      0.014822 -0.020306       6        1   4.988794
5     -0.109538 -0.037573       7        1   4.160586
0     -0.103589 -0.006377       8        1   3.509198
10    -0.042874  0.016049       9        1   3.245806
6      0.085583  0.048188      10        1   2.321383
7      0.090051  0.167063      11        1   2.225310
12     0.085432  0.055225      12        1   1.254993
13     0.020988  0.013558      13        1   1.242979
3      0.106343 -0.129130      14        1   0.414113
14     0.107789 -0.094124      15        1   0.259194, topic_info=     Category         Freq          Term        Total  loglift  logprob
term                                                                   
2275  Default   700.000000        elinor   700.000000  30.0000  30.0000
2294  Default   576.000000      marianne   576.000000  29.0000  29.0000
992   Default  1530.000000          miss  1530.000000  28.0000  28.0000
1426  Default   678.000000     elizabeth   678.000000  27.0000  27.0000
77    Default   829.000000          emma   829.000000  26.0000  26.0000
377   Default  1324.000000          like  1324.000000  25.0000  25.0000
745   Default   953.000000          jane   953.000000  24.0000  24.0000
568   Default   397.000000        madame   397.000000  23.0000  23.0000
802   Default   354.000000     rochester   354.000000  22.0000  22.0000
1052  Default   606.000000           sir   606.000000  21.0000  21.0000
452   Default  1311.000000         think  1311.000000  20.0000  20.0000
1807  Default   438.000000         darcy   438.000000  19.0000  19.0000
2102  Default   291.000000      monsieur   291.000000  18.0000  18.0000
607   Default   930.000000         shall   930.000000  17.0000  17.0000
151   Default  1440.000000          know  1440.000000  16.0000  16.0000
248   Default  1140.000000       thought  1140.000000  15.0000  15.0000
465   Default   424.000000        weston   424.000000  14.0000  14.0000
321   Default  1061.000000           day  1061.000000  13.0000  13.0000
1792  Default   323.000000       bingley   323.000000  12.0000  12.0000
160   Default   941.000000          long   941.000000  11.0000  11.0000
1132  Default   363.000000       fairfax   363.000000  10.0000  10.0000
230   Default   605.000000        sister   605.000000   9.0000   9.0000
221   Default   894.000000          room   894.000000   8.0000   8.0000
1399  Default   340.000000        bennet   340.000000   7.0000   7.0000
120   Default   501.000000       harriet   501.000000   6.0000   6.0000
3180  Default   220.000000    willoughby   220.000000   5.0000   5.0000
49    Default   832.000000          come   832.000000   4.0000   4.0000
146   Default   618.000000          john   618.000000   3.0000   3.0000
76    Default   371.000000         elton   371.000000   2.0000   2.0000
516   Default   221.000000            dr   221.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
3410  Topic15     1.503143           pen    30.520548   2.9445  -6.2035
1587  Topic15     3.714141  mademoiselle   119.415108   2.4849  -5.2989
1589  Topic15     1.394323      millcote    27.958317   2.9570  -6.2786
2218  Topic15     1.540885        pencil    32.764126   2.8984  -6.1787
1143  Topic15     2.070331          gold    53.606728   2.7014  -5.8833
1198  Topic15     1.390185         satin    29.634535   2.8958  -6.2816
1224  Topic15     2.743814         tired    89.577568   2.4696  -5.6017
2674  Topic15     1.387797        silver    31.549011   2.8315  -6.2833
4919  Topic15     1.389965         burns    32.238644   2.8115  -6.2817
106   Topic15  

### Overall Model / Documents

In [26]:
distro = [lda[d] for d in corpus]

In [27]:
distro[0]

[(4, 0.99732375)]

In [28]:
def update(doc):
        d_dist = {k:0 for k in range(0,15)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

In [29]:
df = pd.DataFrame.from_records(new_distro, index=titles)
df.columns = topics

## Part 4: Selecting the Number of Topics
<a id="#p4"></a>

In [30]:
from gensim.models.coherencemodel import CoherenceModel

def compute_coherence_values(dictionary, corpus, path, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    path : path to input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        stream = doc_stream(path)
        model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=id2word, workers=4)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=stream, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [31]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, 
                                                        corpus=corpus, 
                                                        path=path, 
                                                        start=2, 
                                                        limit=40, 
                                                        step=6)

In [32]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

NameError: name 'plt' is not defined

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

## Additional Resources
## Resources

* [Gensim](https://radimrehurek.com/gensim/): Python package for topic modeling, nlp, word vectorization, and few other things. Well maintained and well documented.
* [Topic Modeling with Gensim](http://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#11createthedictionaryandcorpusneededfortopicmodeling): A kind of cookbook for LDA with gensim. Excellent overview, but the you need to be aware of missing import statements and assumed prior knowledge.
* [Chinese Restuarant Process](https://en.wikipedia.org/wiki/Chinese_restaurant_process): That really obscure stats thing I mentioned... 
* [PyLDAvis](https://github.com/bmabey/pyLDAvis): Library for visualizing the topic model and performing some exploratory work. Works well. Has a direct parrell implementation in R as well. 
* [Rare Technologies](https://rare-technologies.com/): The people that made & maintain gensim and a few other libraries.
* [Jane Austen v. Charlotte Bronte](https://www.literaryladiesguide.com/literary-musings/jane-austen-charlotte-bronte-different-alike/)